# Polars
By Borja Barber - Lead instructor DS

En este notebook aprenderás a utilizar **Polars**, una biblioteca de Python para manipular datos tabulares. Polars combina la flexibilidad del ecosistema Python con la velocidad de **Rust** y la eficiencia de los procesamientos *vectorizados* y *columnares*. Según la documentación oficial, Polars es una biblioteca de manipulación de datos orientada a columnas cuyo motor de consulta multi‑hilo escrito en Rust permite paralelizar las operaciones y aprovechar la memoria caché de manera coherente. La API está pensada para ser expresiva y fácil de usar, con nombres familiares para quienes ya han trabajado con otras bibliotecas de datos.

> **Objetivo**: recorreremos desde los conceptos básicos hasta características avanzadas (lazy evaluation, agrupaciones, uniones, transformaciones y manejo de series temporales), proporcionando ejemplos prácticos y explicaciones claras en español.



## Introducción a Polars

Polars es una biblioteca de código abierto para el procesamiento de datos, diseñada para ofrecer un alto rendimiento en un único equipo. Algunas de sus características destacadas son:

- **Rendimiento**: el núcleo de Polars está escrito en Rust, lo que le permite alcanzar rendimientos comparables a bibliotecas en C o C++. Gracias a su motor de consultas multi‑hilo y su procesamiento vectorizado, Polars aprovecha todos los núcleos disponibles de la CPU.
- **Modelo columna**: Polars utiliza almacenamiento en columnas y el formato de memoria **Apache Arrow**, lo que facilita la integración con otras herramientas y permite ejecutar consultas vectorizadas de forma eficiente.
- **API familiar**: si ya conoces `pandas`, te resultará sencilla la transición porque los nombres de métodos y conceptos son similares. Sin embargo, Polars introduce expresiones y contextos que proporcionan un lenguaje declarativo y más optimizado.
- **Instalación flexible**: se puede instalar la versión básica o incluir dependencias opcionales para convertir datos a `pandas` o `numpy`.


## Instalación de Polars

Para instalar Polars con `pip` basta ejecutar en la terminal (desde un entorno virtual si es posible):
```bash
python -m pip install polars
```
Esto instala la versión mínima. Si quieres convertir DataFrames de Polars a `pandas` o `numpy`, añade las dependencias opcionales como sigue【248539729197006†L212-L236】:
```bash
python -m pip install "polars[numpy, pandas]"
```
También puedes instalar todas las funcionalidades con `polars[all]` si planeas usar toda la gama de características.

In [ ]:
# Instalación (esta celda no se ejecuta automáticamente, sirve como referencia)
# Para instalar Polars en tu entorno usa el comando de pip en una terminal:
# !python -m pip install polars[numpy, pandas]

# Una vez instalado, puedes importarlo así:
import polars as pl

# Mostrar la versión instalada de Polars
print(f'Versión de Polars: {pl.__version__}')


## Creación de DataFrames

El núcleo de Polars es el `DataFrame`, una estructura bidimensional de filas y columnas. Cada columna es un `Series` con un tipo de dato concreto. Podemos construir un DataFrame a partir de un diccionario de listas o arrays. En el ejemplo siguiente generamos datos aleatorios con NumPy para ilustrar cómo crear un DataFrame y examinamos su `schema` para ver los tipos de cada columna.


In [ ]:
# Importamos NumPy y Polars
import numpy as np
import polars as pl

# Definimos el número de filas y un generador de números aleatorios
num_rows = 5000
rng = np.random.default_rng(seed=42)  # semilla para reproducibilidad

# Creamos un diccionario con datos aleatorios
datos_edificios = {
    "superficie": rng.exponential(scale=1000, size=num_rows),  # superficie en pies cuadrados
    "anio": rng.integers(low=1995, high=2023, size=num_rows),  # año de construcción
    "tipo_edificio": rng.choice(["A", "B", "C"], size=num_rows),  # categoría
}

# Creamos el DataFrame a partir del diccionario
edificios = pl.DataFrame(datos_edificios)

# Mostramos el esquema (nombres y tipos de columnas)
print(edificios.schema)

# Visualizamos las primeras filas
print(edificios.head())


## Operaciones básicas: `select`, `filter` y `with_columns`

Polars utiliza **expresiones** y **contextos**. Las expresiones (`pl.col`, `pl.lit`, etc.) describen qué se quiere hacer con las columnas, y los contextos (`select`, `filter`, `with_columns`) indican cómo se aplican esas expresiones.

- **`select`** extrae columnas y permite transformarlas encadenando expresiones. Por ejemplo, podemos seleccionar una columna por nombre o aplicar operaciones como ordenar y escalar.
- **`filter`** reduce el DataFrame según una condición. Pasa una expresión booleana y Polars devolverá solo las filas que la cumplan.
- **`with_columns`** crea columnas nuevas o modifica las existentes mediante expresiones.

A continuación se muestran algunos ejemplos.

In [ ]:
# Seleccionamos la columna 'superficie' directamente por nombre
solo_superficie = edificios.select("superficie")
print(solo_superficie.head())

# Seleccionamos la columna usando una expresión y aplicamos transformaciones:
# - ordenamos los valores y
# - los dividimos entre 1000 para convertir de pies cuadrados a miles de pies cuadrados
superficie_ordenada = edificios.select((pl.col("superficie").sort() / 1000).alias("superficie_miles"))
print(superficie_ordenada.head())

# Filtramos edificios construidos después de 2015 y calculamos el mínimo año para comprobar
despues_2015 = edificios.filter(pl.col("anio") > 2015)
print(despues_2015.shape)
print(despues_2015.select(pl.col("anio").min().alias("anio_minimo")))

# Creamos una nueva columna con el logaritmo de la superficie usando with_columns
edificios_log = edificios.with_columns([
    (pl.col("superficie").log10()).alias("log_superficie")
])
print(edificios_log.head())


## Agrupaciones y agregaciones (`group_by`)

El contexto `group_by` agrupa filas según uno o varios campos y permite aplicar operaciones de agregación. Puedes calcular medias, medianas, conteos y otras funciones para cada grupo.

En el ejemplo de los edificios, agruparemos por `tipo_edificio` y calcularemos:

- La **media** de la superficie.
- La **mediana** del año de construcción.
- El **número** de edificios en cada categoría.


In [ ]:
# Agrupamos por 'tipo_edificio' y calculamos estadísticas
resumen_tipo = edificios.group_by("tipo_edificio").agg([
    pl.col("superficie").mean().alias("media_superficie"),
    pl.col("anio").median().alias("mediana_anio"),
    pl.count().alias("conteo")
])
print(resumen_tipo)


## Uniones (`join`) y concatenación

Polars permite unir tablas de distintas formas (inner, left, outer, cross) y concatenar DataFrames vertical u horizontalmente. Las uniones se especifican mediante el método `.join()`, indicando las columnas de unión y el tipo de join.

A continuación creamos dos DataFrames de muestra y realizamos un join interno para combinar la información.

In [ ]:
# Creamos dos DataFrames de ejemplo
df_izq = pl.DataFrame({
    "id": [1, 2, 3],
    "nombre": ["Ana", "Beto", "Carla"],
})

df_der = pl.DataFrame({
    "id": [2, 3, 4],
    "ciudad": ["Valencia", "Madrid", "Sevilla"],
})

# Realizamos un inner join usando la columna 'id'
df_join = df_izq.join(df_der, on="id", how="inner")
print(df_join)

# Concatenamos verticalmente (apilamos filas)
df_concat = pl.concat([df_izq, df_der.select(["id"]).with_columns([
    pl.lit(None).alias("nombre")
])], how="vertical")
print(df_concat)


## Transformaciones de forma: pivote y derribar (`pivot`, `melt`)

Las tablas no siempre vienen en el formato deseado. Polars ofrece operaciones para reorganizar los datos:

- **`pivot`** convierte una columna de valores en nuevas columnas, permitiendo crear tablas anchas a partir de datos longitudinales.
- **`melt`** (o *unpivot*) realiza la operación inversa: transforma columnas en filas.

A continuación un ejemplo sencillo.

In [ ]:
# DataFrame de ventas mensuales por producto
ventas = pl.DataFrame({
    "producto": ["A", "A", "B", "B"],
    "mes": ["Enero", "Febrero", "Enero", "Febrero"],
    "ventas": [100, 150, 80, 120],
})

# Pivotamos para que cada producto tenga una columna por mes
ventas_pivot = ventas.pivot(values="ventas", index="producto", columns="mes")
print(ventas_pivot)

# Derribamos (melt) las columnas de vuelta a formato largo
ventas_unpivot = ventas_pivot.melt(id_vars="producto", value_vars=["Enero", "Febrero"])
print(ventas_unpivot)


## Manejo de series temporales

Polars incluye soporte para datos de fecha y hora. Puedes convertir cadenas a fechas con `pl.col().str.strptime`, combinar filtrado y agrupación por ventanas de tiempo y realizar re‐muestreo.

En el siguiente ejemplo creamos un DataFrame de temperaturas por hora, lo convertimos a un índice temporal y luego calculamos medias diarias.

In [ ]:
# Generamos fechas por hora durante tres días
fechas = pl.date_range(start="2023-01-01", end="2023-01-03", interval="1h")

# Simulamos valores de temperatura
temperaturas = pl.Series(np.random.normal(loc=20, scale=5, size=len(fechas)))

# Creamos el DataFrame
clima = pl.DataFrame({
    "fecha": fechas,
    "temperatura": temperaturas,
})

# Convertimos la columna de fecha a tipo Date (ya viene como DateTime) y re‐muestreamos a diario
clima_resampled = (
    clima
    .group_by(pl.col("fecha").dt.date().alias("dia"))
    .agg(pl.col("temperatura").mean().alias("temp_media"))
)

print(clima_resampled)


## API *Lazy* y optimización

Una de las características más potentes de Polars es su **API perezosa** (*lazy API*). En lugar de ejecutar inmediatamente cada operación, Polars construye un grafo de consultas que se optimiza antes de ser evaluado. Esto permite detectar errores de esquema antes de leer los datos y aplicar optimizaciones como el *predicate pushdown*, que aplica filtros cuanto antes para reducir el tamaño intermedio.

En el siguiente ejemplo creamos un `LazyFrame`, definimos una serie de transformaciones y filtros, exploramos el plan de consulta con `.explain()` y finalmente ejecutamos la consulta con `.collect()`.


In [ ]:
# Convertimos nuestro DataFrame 'edificios' en un LazyFrame
edificios_lazy = edificios.lazy()

# Definimos una consulta: calculamos el precio por superficie, filtramos por condiciones y ordenamos
consulta_lazy = (
    edificios_lazy
    .with_columns((pl.col("superficie") / 100).alias("superficie_100"))
    .filter(pl.col("superficie_100") > 10)
    .filter(pl.col("anio") < 2010)
)

# Mostramos el plan de consulta (string)
print(consulta_lazy.explain())

# Ejecutamos la consulta con collect para obtener un DataFrame
resultado = consulta_lazy.collect()
print(resultado.head())


## Interoperabilidad con pandas y NumPy

Polars puede trabajar en conjunto con otras bibliotecas. Si instalas las dependencias opcionales, puedes convertir un DataFrame de Polars a un DataFrame de `pandas` o a un array de `numpy`.

Esto resulta útil cuando necesitas utilizar funciones específicas de otras bibliotecas, visualización o compatibilidad con código existente.

In [ ]:
# Convertimos el DataFrame 'edificios' a pandas y a NumPy
df_pandas = edificios.to_pandas()
print(type(df_pandas))  # <class 'pandas.core.frame.DataFrame'>

# Convertimos una columna a array de NumPy
superficie_np = edificios.select("superficie").to_numpy().flatten()
print(type(superficie_np), superficie_np[:5])


## Conclusión

Has recorrido los fundamentos de **Polars**, desde la creación y exploración de DataFrames hasta operaciones avanzadas como agrupaciones, uniones y transformaciones. También exploraste la potente **API perezosa**, que construye un plan de consulta optimizado antes de ejecutarlo. Polars aprovecha al máximo los recursos de tu equipo al ejecutar consultas en paralelo y utilizar almacenamiento columnar.

Para seguir profundizando te recomendamos consultar la documentación oficial y experimentar con tus propios conjuntos de datos. ¡Ahora tienes las bases para convertirte en un maestro de Polars!